### 由于Apriori关联规则算法无法处理连续型数值变量，所以使用聚类算法对各个证型系数进行离散化处理
> p_188

In [1]:
import pandas as pd
# 导入KMeans聚类
from sklearn.cluster import KMeans

In [2]:
# 带聚类文件
datafile = 'E:\\NLP\\Python_Data_Analysis_And_Mining_Actual_Combat\\chapter8\\demo\\data\\data.xls'
# 离散化处理之后的保存文件
processedfile = 'E:\\NLP\\Python_Data_Analysis_And_Mining_Actual_Combat\\chapter8\\demo\\tmp\\data_processed.xls'

In [3]:
# 各证型系数标签
typelabel = {
    '肝气郁结证型系数': 'A',
    '热毒蕴结证型系数': 'B',
    '冲任失调证型系数': 'C',
    '气血两虚证型系数': 'D',
    '脾胃虚弱证型系数': 'E',
    '肝肾阴虚证型系数': 'F'
}
# 需要聚类的类别
k = 4

In [6]:
# 读取数据
data = pd.read_excel(datafile)
# 各证型系数
keys = list(typelabel.keys())
print(keys)
# 结果储存容器
result = pd.DataFrame()
print(result)
test = pd.Series()
print(test)

['肝气郁结证型系数', '热毒蕴结证型系数', '冲任失调证型系数', '气血两虚证型系数', '脾胃虚弱证型系数', '肝肾阴虚证型系数']
Empty DataFrame
Columns: []
Index: []
Series([], dtype: float64)


In [ ]:
# 依次对每个证型系数进行聚类
for i in range(len(keys)):
    # 调用K-Means算法进行聚类离散化
    print('当前进行 %s 的聚类' %(keys[i]))
    # n_jobs = 4 并行数，一般等于cpu数就好
    kmodel = KMeans(n_clusters = k, n_jobs = 4)
    # 训练 (取对应证型系数的那一列并转为矩阵形式)
    kmodel.fit(data[[keys[i]]].as_matrix())
    
    # 聚类中心
    r1 = pd.DataFrame(kmodel.cluster_centers_, columns = [typelabel[keys[i]]])
    # 分类统计 看上面的 test 和 result 打印的结果就明白了
    r2 = 